In [ ]:
# There's a gap between the oldest images available via takeout and the
# oldest actually still stored by groupme.  Rectify this.

import os

In [ ]:
def image_url_to_gallery_filename(chat_id, message_id, image_url):
    res, ext, id = image_url.split('/')[-1].split('.')
    return os.path.join('./data', str(chat_id), 'gallery', f'{message_id}_{res}.{id}.{ext}')

def video_url_to_gallery_filename(chat_id, message_id, video_url):
    id, res, ext = video_url.split('/')[-1].split('.')
    return os.path.join('./data', str(chat_id), 'gallery', f'{message_id}_{id}.{res}.{ext}')

In [ ]:
# for each dir in ./data, scrape the message.json file for all attachments

import json

missing_attachments = []
found = 0
missing = 0

for chat in os.listdir('./data'):
    chat_dir = os.path.join('./data', chat)
    if not os.path.isdir(chat_dir):
        continue
    message_json = os.path.join(chat_dir, 'message.json')
    if not os.path.isfile(message_json):
        continue
    with open(message_json) as f:
        messages = json.load(f)
        for message in messages:
            for attachment in message['attachments'] or []:
                filename = "unknown"
                if attachment['type'] == 'image':
                    # attachment_urls.append(attachment['url'])
                    filename = image_url_to_gallery_filename(chat, message['id'], attachment['url'])
                elif attachment['type'] == 'video':
                    filename = video_url_to_gallery_filename(chat, message['id'], attachment['url'])
                else:
                    continue
                if os.path.isfile(filename):
                    print(filename)
                    found += 1
                else:
                    print(f'Missing {attachment}')
                    missing_attachments.append({"original": attachment, "expected_filename": filename})
                    missing += 1
print(f'Found {found} images, missing {missing}')

In [ ]:
import requests

succeeded = 0

for missing in missing_attachments:
    if "original" not in missing.keys() or "url" not in missing["original"].keys():
        print(f"Missing expected key in {missing}")
        continue
    print(missing["original"]["url"], "->", missing["expected_filename"])
    resp = requests.get(missing["original"]['url'])
    if resp.status_code == 200:
        with open(missing["expected_filename"], 'wb') as handler:
            handler.write(resp.content)
            succeeded += 1
            print("\tDone")
    else:
        print(f"\tError {resp.status_code}")